<a href="https://colab.research.google.com/github/hood-boi/world-news-chatbot/blob/master/Data_Collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langdetect

     |████████████████████████████████| 1.0MB 5.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/ec/0c/a9/1647275e7ef5014e7b83ff30105180e332867d65e7617ddafe
Successfully built langdetect


In [2]:
import requests
import json
import time
import string
import nltk
from langdetect import detect
nltk.download('words')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [0]:
printable = set(string.printable)

def cleanString(text):
    printable = set(string.printable)
    text_i = ''.join(filter(lambda x: x in printable, text))
    return text_i;

In [6]:
! wc '/content/gdrive/My Drive/Chatbot/rworldnews.txt'

  248952 11261157 65389928 /content/gdrive/My Drive/Chatbot/rworldnews.txt


In [7]:
subreddit = "AskReddit"

def updateListingDict(postsJSON, postsDICT):
    chunk_size = len(postsJSON["data"]["children"])
    posts = postsJSON["data"]["children"]
    
    for i in range(chunk_size): #Loop through each reddit post
        r_post_title = posts[i]["data"]["title"];
        r_post_title = " ".join(r_post_title.split());
        r_post_title = r_post_title.replace('\n', ' ');
        r_post_title = r_post_title.replace('\t', ' ');
        r_post_title = r_post_title.replace('\'', '');
        r_post_title = r_post_title.replace('\"', '');

        postsDICT[posts[i]["data"]["id"]] = cleanString(r_post_title);

after = '';
listing_dict = {};

for i in range(0, 10):
    if(after):
        print("[AFTER] = ", after)
        r_worldnews = requests.get('https://www.reddit.com/r/'+ subreddit +'/top.json?t=all&limit=100&after='+after, 
                                headers = { 'User-agent' : 'ChatBot', 
                                            'Accept' : '*/*',
                                            'Cache-Control' : 'no-cache'
                                          });
        if(r_worldnews.status_code == 200):
            postsJSON = r_worldnews.json()
            updateListingDict(postsJSON, listing_dict);
            after = postsJSON["data"]["after"];
    else:
        print("[AFTER] = NONE")
        r_worldnews = requests.get('https://www.reddit.com/r/'+ subreddit +'/top.json?t=all&limit=100', 
                                headers = { 'User-agent' : 'ChatBot', 
                                            'Accept' : '*/*',
                                            'Cache-Control' : 'no-cache' 
                                          });
        if(r_worldnews.status_code == 200):
            postsJSON = r_worldnews.json()
            updateListingDict(postsJSON, listing_dict);
            #print("[GET] request successful")
            after = postsJSON["data"]["after"];
            
            

[AFTER] = NONE
[AFTER] =  t3_a7nwe2
[AFTER] =  t3_at2yq2
[AFTER] =  t3_afqynk
[AFTER] =  t3_bqc3ib
[AFTER] =  t3_c8zwj2
[AFTER] =  t3_9tm8ku
[AFTER] =  t3_8p0pat
[AFTER] =  t3_c27tbb
[AFTER] =  t3_aovbo4


In [8]:
print(len(listing_dict))

for key, value in listing_dict.items():
    print(key, value)
    break;

994
ablzuq People who havent pooped in 2019 yet, why are you still holding on to last years shit?


In [9]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
file_handle = open('/content/gdrive/My Drive/Chatbot/r'+subreddit+'.txt', 'a+')

#Expecting array of JSON children
def recursePostComments(context, commentsJSON):
   
    size = len(commentsJSON)
    for i in range(0, size):
        #print(commentsJSON[i])
        if('data' in commentsJSON[i].keys()):
            if('body' in commentsJSON[i]['data'].keys()):
                reply = commentsJSON[i]['data']['body'];
                if("[deleted]" in reply or "[removed]" in reply):
                    continue;
                else:
                    reply = reply.replace('\n', ' ');
                    reply = reply.replace('\t', ' ');
                    reply = reply.replace('\'', '');
                    reply = reply.replace('\"', '');
                    reply = reply.replace('&gt', '');
                    reply = reply.replace('&lt', '');
                    #print("Context : ", context);
                    
                    if (len(reply) > 500):
                        continue;
                        #print("\tLen :", len(reply));
                    #    t_reply = summarizeText(reply);
                    #    if (len(t_reply) > 0):
                    #        reply = t_reply;
                    #print("\tReply :", reply);
                    
                    reply = cleanString(reply);
                    reply = " ".join(reply.split());
                                        
                    
                    if(reply.isspace() or context.isspace()):
                        print("Error : ", reply, ", ", context)
                        continue;                    
                    else:
                        line_to_write = "{}\t{}\n".format(reply.strip(), context.strip());
                        file_handle.write(line_to_write)
                    
                    if('replies' in commentsJSON[i]['data'].keys()):
                        if(commentsJSON[i]['data']['replies']):
                            if('data' in commentsJSON[i]['data']['replies'].keys()):
                                if(commentsJSON[i]['data']['replies']['data']):
                                    if('children' in commentsJSON[i]['data']['replies']['data'].keys()):
                                        recursePostComments(reply, commentsJSON[i]['data']['replies']['data']['children']);
            else:
                continue;
        else:
            continue;
    return;


In [0]:
! wc '/content/gdrive/My Drive/Chatbot/rworldnews.txt'

  248686 11250503 65328320 /content/gdrive/My Drive/Chatbot/rworldnews.txt


In [0]:
for key, value in listing_dict.items():
    print(key, value)
    time.sleep(2)
    r_comments = requests.get('https://www.reddit.com/r/'+ subreddit +'/comments/' + key +'.json?limit=1000&sort=confidence&depth=3', 
                                headers = { 'User-agent' : 'ChatBot', 
                                            'Accept' : '*/*',
                                            'Cache-Control' : 'no-cache' 
                                          });
    r_comments_json = r_comments.json();
    context = value;
    commentsJSON = r_comments_json[1]["data"]["children"]
    recursePostComments(context, commentsJSON)
    
file_handle.close()

In [0]:
#https://stackoverflow.com/questions/8689795/how-can-i-remove-non-ascii-characters-but-leave-periods-and-spaces-using-python
    

In [0]:
import string
print(string.printable)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	



In [22]:
! cd '/content/gdrive/My Drive/Chatbot/' ; ls r*.txt | xargs -L 1 wc

  325392 16109903 87161691 rAskReddit.txt
  337107 15033769 85689554 rnews.txt
  339541 12756240 72687966 rnottheonion.txt
   68529  3350392 19319193 routoftheloop.txt
  248952 11261157 65389928 rworldnews.txt
